In [1]:
import numpy as np

from sudoku_stuff import *

# Resolviendo Sudokus con Local Beam
Inteligencia Artificial - Facundo A. Lucianna - CEIA - FIUBA

En las notebooks anteriores, intentamos resolver sudokus utilizando los algoritmos de gradiente descendente y simulated annealing.

Ahora, a diferencia de los métodos anteriores, donde se comenzaba la búsqueda desde un único estado inicial, la **búsqueda local beam** guarda la información de **k estados** y realiza la búsqueda de manera independiente sobre ellos.

Resolvamos este problema:

<div>
<img src="./sudoku_7.png" width="300"/>
</div>

El diccionario de **celdas fijas** quedaría de la siguiente forma:

In [2]:
fixed_squares = {
    'A1': 3, 'A3': 4, 'A4': 5, 'A5': 6, 'A7': 9,
    'B1': 1, 'B2': 8, 'B3': 5, 'B6': 9, 'B7': 7,
    'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D2': 2, 'D5': 1, 'D8': 4, 'D9': 9,
    'E2': 4, 'E3': 9, 'E5': 5, 
    'F3': 1, "F4": 9, "F5": 8, "F7": 6, "F8": 7,
    'G1': 4, 'G2': 9, 'G5': 3, 'G9': 7, 
    'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H9': 6,
    'I8': 8,
}

Y la solución que tenemos, que vamos a usar para verificar al final de todo es:

In [3]:
solution = {
    'A1': 3, 'A2': 7, 'A3': 4, 'A4': 5, 'A5': 6, 'A6': 1, 'A7': 9, 'A8': 2, 'A9': 8,
    'B1': 1, 'B2': 8, 'B3': 5, 'B4': 4, 'B5': 2, 'B6': 9, 'B7': 7, 'B8': 6, 'B9': 3,
    'C1': 9, 'C2': 6, 'C3': 2, 'C4': 3, 'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D1': 8, 'D2': 2, 'D3': 7, 'D4': 6, 'D5': 1, 'D6': 3, 'D7': 5, 'D8': 4, 'D9': 9,
    'E1': 6, 'E2': 4, 'E3': 9, 'E4': 2, 'E5': 5, 'E6': 7, 'E7': 8, 'E8': 3, 'E9': 1,
    'F1': 5, 'F2': 3, 'F3': 1, 'F4': 9, 'F5': 8, 'F6': 4, 'F7': 6, 'F8': 7, 'F9': 2,
    'G1': 4, 'G2': 9, 'G3': 6, 'G4': 8, 'G5': 3, 'G6': 2, 'G7': 1, 'G8': 5, 'G9': 7,
    'H1': 2, 'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H7': 3, 'H8': 9, 'H9': 6,
    'I1': 7, 'I2': 5, 'I3': 3, 'I4': 1, 'I5': 9, 'I6': 6, 'I7': 2, 'I8': 8, 'I9': 4,
}

In [4]:
print_state(solution)

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


## Implementado búsqueda por Local Beam

Este algoritmo comienza con k estados que se inician al azar. En cada paso, se generan sucesores de esos **k estados**. Si alguno de los sucesores cumple con el objetivo, el algoritmo termina. En caso contrario, se seleccionan los k mejores sucesores de la lista. A simple vista, podría parecer lo mismo que ejecutar k veces el algoritmo de gradiente descendente, pero la diferencia radica en que, entre los procesos de búsqueda, hay un intercambio de información:

*Si un estado genera varios sucesores buenos y los otros k-1 estados generan sucesores malos, el efecto es que el primer estado "abandona" la búsqueda de los demás y se queda con los sucesores que generó.*

Veamos la implementación, leyendo cada comentario con atención:

In [5]:
def local_beam_search(initial_generation, fixed_squares, max_iterations=100):
    """
    Realiza la optimización del Sudoku utilizando búsqueda por Local Beam.

    Args:
        initial_generation (list): Lista con estados iniciales del Sudoku.
        fixed_squares (dict): Diccionario que contiene las casillas fijas del Sudoku.
        max_iterations (int, optional): El número máximo de iteraciones permitidas.
                                        Por defecto es 100.

    Returns:
        dict: El mejor estado encontrado después de la optimización.
        float: El costo del mejor estado encontrado
        int: Numero indicando en que generación se encontró el mejor resultado
    """
    # Inicializamos para poder arrancar    
    best_state = initial_generation[0]
    best_cost = cost_function(best_state)
    best_iteration = 0
    no_changes = 0

    number_population = len(initial_generation)
    current_population = initial_generation.copy()

    # Iteramos hasta max_iterations
    for iteration in range(max_iterations):
        no_changes += 1

        # Para cada uno de la generación, calculamos su función de costo
        actual_cost_list = []
        for index in range(number_population):
            state = current_population[index]
            cost = cost_function(state)
            actual_cost_list.append(cost)
            if cost < best_cost:
                no_changes = 0
                best_state = state
                best_cost = cost
                best_iteration = iteration
                print(f"El mejor costo es: {best_cost} en la iteración {best_iteration}")

        # Si es solución, terminamos.
        # Tambien si pasamos varias iteraciones sin cambios, nos trabamos
        if best_cost == 0 or no_changes > 5:
            return best_state, best_cost, best_iteration
            
        # Obtenemos a todos los vecinos posibles y sus costos
        all_neib = []
        all_cost = []
        for index, state in enumerate(current_population):
            actual_neib = return_neib_states(state, fixed_squares)
            all_neib += actual_neib
            all_cost += [cost_function(state_neib) for state_neib in actual_neib]

        # Ordenamos a los vecinos en función del costo
        index_neib_list = sorted(range(len(all_cost)), key=lambda x: all_cost[x])
        all_neib = [all_neib[k] for k in index_neib_list]

        # Nos quedamos con los k de la generación
        current_population = all_neib[:number_population]

    # Si terminamos las iteraciones, retornamos el mejor resultado encontrado
    return best_state, best_cost, best_iteration

Ahora, veamos si podemos encontrar la solución en una única ejecución. Para ello, vamos a llamar a una función llamada `execute_search_evolution()`, a la cual le pasamos la implementación del algoritmo. Esta función inicializa varios estados de sudoku al azar, aplica la búsqueda y verifica si se ha alcanzado una solución o no.

In [6]:
from processing import execute_search_evolution

In [7]:
solution_bool, best_state, best_generation, _ = execute_search_evolution(0, local_beam_search, fixed_squares, number_generation_initial=20)

El mejor costo es: 5.699999999999999 en la iteración 0
El mejor costo es: 5.299999999999999 en la iteración 1
El mejor costo es: 5.0 en la iteración 2
El mejor costo es: 4.7 en la iteración 3
El mejor costo es: 4.3999999999999995 en la iteración 4
El mejor costo es: 4.1000000000000005 en la iteración 5
El mejor costo es: 3.800000000000001 en la iteración 6
El mejor costo es: 3.600000000000001 en la iteración 7
El mejor costo es: 3.400000000000001 en la iteración 8
El mejor costo es: 3.2000000000000006 en la iteración 9
El mejor costo es: 3.0000000000000004 en la iteración 10
El mejor costo es: 2.8000000000000003 en la iteración 11
El mejor costo es: 2.6000000000000005 en la iteración 12
El mejor costo es: 2.4000000000000004 en la iteración 13
El mejor costo es: 2.2 en la iteración 14
El mejor costo es: 2.1 en la iteración 15
El mejor costo es: 1.9000000000000001 en la iteración 16
El mejor costo es: 1.8 en la iteración 17
El mejor costo es: 1.6000000000000003 en la iteración 18
El mejo

In [8]:
print(f"Valor del mejor costo encontrado: {cost_function(best_state)}, el mejor estado se encontró en la {best_generation} generación")

print("Mejor estado encontrado:")
print_state(best_state)

Valor del mejor costo encontrado: 0.0, el mejor estado se encontró en la 32 generación
Mejor estado encontrado:
*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


In [9]:
print("El estado encontrado es solución?")
if solution_bool:
    print("El estado que encontramos verifica que realmente es la solución")
else:
    print("El estado que encontramos no es solución")

El estado encontrado es solución?
El estado que encontramos verifica que realmente es la solución


Vemos que el algoritmo está encontrando la **solución**. Para el caso del Sudoku, esta estrategia de iniciar varias búsquedas en paralelo y abandonar aquellos caminos infructuosos da buenos resultados.

Como prueba, vamos a ejecutar la búsqueda 100 veces y verificar cuántas veces llega a una solución. Para acelerar el proceso, aprovecharemos que tenemos **CPUs multinúcleo**.

Para ello, vamos a llamar a la función `parallel_sudoku_search()`, a la cual le pasaremos la función de búsqueda y el número de iteraciones que queremos realizar, indicandole que estamos usando un algoritmo de búsqueda que usa multiple estados.

In [10]:
from processing import parallel_sudoku_search

# Debemos llamar a la función de busqueda desde un archivo .py sino los threads no pueden recibir a la función desde la notebook directamente.
from search_methods import local_beam_search

In [11]:
results = parallel_sudoku_search(local_beam_search, fixed_squares, max_iterations=100, generation_method=True)

  0%|          | 0/100 [00:00<?, ?it/s]

Veamos si algún proceso encontró la solución:

In [12]:
show_solution = True
for res in results:
    # Acá nos devuelve el booleano de si encontró la solución o no
    is_solution = res[0]
    # Este es el últimos estado encontrado en esta iteración
    last_state = res[1]
    # Este es el estado desde donde partio
    initial_state = res[2]
    # Este es el identificador de cual iteración se obtuvo la solución
    process_id = res[-1]

    if is_solution:
        if show_solution:
            print_state(last_state)
            show_solution = False
        print(f"En la iteración {process_id} se encontró la solución.")

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*
En la iteración 14 se encontró la solución.
En la iteración 15 se encontró la solución.
En la iteración 18 se encontró la solución.
En la iteración 21 se encontró la solución.
En la iteración 29 se encontró la solución.
En la iteración 32 se encontró la solución.
En la iteración 34 se encontró la solución.
En la iteración 39 se encontró la solución.
En la iteración 41 se encontró la solución.
En la iteración 44 se encontró la solución.
En la iteración 46 se encontró la solución.
En la iteración 48 se encontró la solución.
En la iteración 71 se encontró la solución.
En la iterac

Aquí verificamos que, en efecto, **Local Beam** da resultados muy positivos. En muchas de las 100 ejecuciones, encontró la solución.